In [1]:
import numpy as np
import pandas as pd
import pickle
import sys

input_data = pd.read_pickle('../dados/df_ata_sentimento.pkl') 
#input_data = pd.read_json('../dados/df_ata_sentimento.json', orient='records', lines=True)
#input_data['Data'] = pd.to_datetime(input_data['Data']).dt.date
#input_data.tail()
# A obtenção de dados da coluna Selic 6m dixa lacunas, limpar os dados:
input_data = input_data.dropna(subset=['Sentimento','Texto'])
input_data = input_data[~(input_data['Texto'].str.strip().isna() | (input_data['Texto'].str.strip() == ''))]
input_data['Texto'] = input_data['Texto'].str.lower()

input_data.sample(5)




,Data,Texto,Selic,IPCA,Selic (6m),Sentimento
32,2002-02-20,sumário atividade econômica ambiente externo p...,19.00,7.51,18.50,dovish
29,2001-11-21,sumário atividade econômica ambiente externo p...,19.00,7.61,18.75,dovish
110,2009-10-21,sumário evolução recente da economia avaliação...,8.75,4.17,8.75,neutral
229,2024-09-18,a atualização da conjuntura econômica e do cen...,10.50,4.42,13.25,hawkish
45,2003-02-19,sumário atividade econômica ambiente externo p...,25.50,15.85,26.50,hawkish


In [2]:
base_data = input_data[['Texto','Sentimento']]
base_data.sample(10)
filename_root = 'sentimento_6m_pretrained_'
subset_dir = '../../subsets/'
basepickle = subset_dir + filename_root + '__base.pkl'
base_data.to_pickle(basepickle)
pd.read_pickle(basepickle).sample(5)


,Texto,Sentimento
103,sumário evolução recente da economia avaliação...,dovish
59,sumário evolução recente da inflação avaliação...,dovish
25,sumário atividade econômica preços ambiente ex...,neutral
81,sumário evolução recente da economia avaliação...,dovish
34,sumário atividade econômica ambiente externo p...,dovish


In [3]:
from sklearn.preprocessing import LabelEncoder
#tratamento dos dados para modelo
training_data = base_data.copy()
#Encoding do label alfanumérico
label_encoder = LabelEncoder()
training_data.loc[:, 'sent_encoded'] = label_encoder.fit_transform(training_data['Sentimento'])


# Criando 3 colnuas com hot encoding
one_hot = pd.get_dummies(training_data['Sentimento'], prefix='sent')
training_data = pd.concat([training_data, one_hot], axis=1)

# Renomeando colunas
training_data = training_data.rename(columns={'Texto': 'text'})
training_data = training_data.rename(columns={'Sentimento': 'sent'})

# Display the result
print(training_data.sample(5))



                                                  text     sent  sent_encoded  \
93   sumário evolução recente da economia avaliação...   dovish             0   
32   sumário atividade econômica ambiente externo p...   dovish             0   
210  a atualização da conjuntura econômica e do cen...  hawkish             1   
4    sumário demanda e oferta agregadas ambiente ex...  neutral             2   
199  a atualização da conjuntura econômica e do cen...  hawkish             1   

     sent_dovish  sent_hawkish  sent_neutral  
93          True         False         False  
32          True         False         False  
210        False          True         False  
4          False         False          True  
199        False          True         False  


In [4]:
#Com pretrino não será usada tokenização.
# Concatenando todas as atas

#all_text = ' '.join(training_data['text'].tolist())

#words = pd.Series(all_text.split())

# Contando as palavras únicas

#word_count = words.nunique()

#print(f"Palavras únicas na base: {word_count}")


In [5]:

# Switch de qual label vai ser estudado.
label_set = 'sent_encoded'
#label_set = 'sent_dovish'
#label_set = 'sent_hawkish'
#label_set = 'sent_neutral'

#Globais de processsamento
rnd_seed = 42
num_epochs = 50
num_batches = 16
token_count = 30000
padding_size = 80000
val_split = 0.2


In [6]:
#Não será usado token nem pad para pre embed
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizando para 30000 palavras, o dobro do medido na base
#tokenizer = Tokenizer(num_words=token_count, oov_token='<OOV>')
#tokenizer.fit_on_texts(training_data['text'])

# Save the tokenizer to a file

#token_filename = subset_dir + filename_root + '__tokenizer.pkl'

#with open(token_filename, 'wb') as file:
#    pickle.dump(tokenizer, file)

# Preenchendo com 80000 caracterse, englobando o tamanho de todas as atas com alguma folga

#sequences = tokenizer.texts_to_sequences(training_data['text'])
#padded_sequences = pad_sequences(sequences, maxlen=padding_size)

#print(padded_sequences)

#print("{:.2f}".format(sys.getsizeof(padded_sequences)/(1024*1024)),'MB')
#print(padded_sequences[0])

In [7]:


# Splitting the data into features (X) and labels (y)
X = training_data['text'].values
y = training_data[label_set].values

print(str(X[:5])[:100])
print(str(y[:5])[:100])

['sumário preços e nível de atividade agregados monetários e crédito finanças públicas balanço de pa
[0 0 0 0 2]


In [8]:
#Salvando X e y completos para eventual uso
X_filename = subset_dir + filename_root + '_' + label_set + '_X_PRETRAINED_full.npy'
np.save(X_filename, X)

y_filename = subset_dir + filename_root + '_' + label_set + '_y_PRETRAINED_full.npy'
np.save(y_filename, y)



In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rnd_seed)

#Salvando conjuntos de treino e teste para uso posterior

X_test_filename = subset_dir + filename_root +  '_' + label_set + '_X_PRETRAINED_test.npy'
np.save(X_test_filename, X_test)
X_train_filename = subset_dir + filename_root +  '_' + label_set + '_X_PRETRAINED_train.npy'
np.save(X_train_filename, X_train)

y_test_filename = subset_dir + filename_root + '_' + label_set + '_y_PRETRAINED_test.npy'
np.save(y_test_filename, y_test)
y_train_filename = subset_dir + filename_root + '_' + label_set + '_y_PRETRAINED_train.npy'
np.save(y_train_filename, y_train)

print(str(X_train[:5])[:100])
print(y_train[:5])
print(str(X_test[:5])[:100])

print(y_test[:5])



['sumário evolução recente da economia avaliação prospectiva das tendências da inflação implementaçã
[1 0 1 1 1]
['sumário demanda e oferta agregadas ambiente externo preços avaliação prospectiva das tendências da
[0 0 1 0 1]


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Layer

# Define a custom Keras layer to wrap the USE layer
class USELayer(Layer):
    def __init__(self, **kwargs):
        super(USELayer, self).__init__(**kwargs)
        self.use_layer = hub.KerasLayer(
            "https://tfhub.dev/google/universal-sentence-encoder/4",
            input_shape=[],
            dtype=tf.string,
            trainable=True
        )

    def call(self, inputs):
        return self.use_layer(inputs)

# Define the model using Functional API
inputs = tf.keras.Input(shape=(), dtype=tf.string)  # Input layer for strings
x = USELayer()(inputs)  # Apply custom USE layer
x = tf.keras.layers.Reshape((-1, 512))(x)  # Reshape to (batch_size, 1, 512) for LSTM
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)
x = tf.keras.layers.Dense(24, activation='relu')(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)  # 3 classes

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Define checkpoint callback
ckptname = '../../' + filename_root + '_' + label_set + '_' + str(num_epochs) + '_CHKP_drlee_ata.keras'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    ckptname,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False
)

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # For integer labels (0, 1, 2)
    metrics=['accuracy']
)

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, validation_split=val_split, callbacks=[checkpoint_cb])

/opt/anaconda3/envs/tf2.20/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Epoch 1/50


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')